In [1]:
import pandas as pd
import numpy as np
import re

import time # importing time module

## Load IMDB Titles

In [2]:
imdb_df=pd.read_csv('output/IMDB_movies.csv')

imdb_df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,101 Dalmatians,1996,G,27 Nov 1996,103,"Adventure, Comedy, Crime",Stephen Herek,"Dodie Smith, John Hughes","Glenn Close, Jeff Daniels, Joely Richardson","English, Spanish","United States, United Kingdom",Nominated for 1 BAFTA Film Award3 wins & 10 no...,49.0,5.7,"109,762",tt0115433
1,102 Dalmatians,2000,G,22 Nov 2000,100,"Adventure, Comedy, Family",Kevin Lima,"Dodie Smith, Kristen Buckley, Brian Regan","Glenn Close, Gérard Depardieu, Ioan Gruffudd",English,"United States, United Kingdom",Nominated for 1 Oscar. 1 win & 8 nominations t...,35.0,4.9,"37,045",tt0211181
2,1492: Conquest of Paradise,1992,PG-13,09 Oct 1992,154,"Adventure, Biography, Drama",Ridley Scott,Rose Bosch,"Gérard Depardieu, Armand Assante, Sigourney We...","English, Spanish, Latin, Arabic","United Kingdom, France, Spain",2 nominations,47.0,6.4,"29,990",tt0103594
3,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127,"Adventure, Drama, Family",Richard Fleischer,"Earl Felton, Jules Verne","Kirk Douglas, James Mason, Paul Lukas",English,United States,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,"33,152",tt0046672
4,25th Hour,2002,R,10 Jan 2003,135,Drama,Spike Lee,David Benioff,"Edward Norton, Barry Pepper, Philip Seymour Ho...",English,United States,4 wins & 17 nominations,68.0,7.6,"176,763",tt0307901


In [3]:
imdb_df = imdb_df[['Title','Year','Released']].copy()
#imdb_df.rename(columns={'Title': 'movie_title', 'Year': 'year', 'Released': 'release_date'}, inplace = True)
imdb_df.head()

,Title,Year,Released
0,101 Dalmatians,1996,27 Nov 1996
1,102 Dalmatians,2000,22 Nov 2000
2,1492: Conquest of Paradise,1992,09 Oct 1992
3,"20,000 Leagues Under the Sea",1954,20 Jul 1955
4,25th Hour,2002,10 Jan 2003


In [4]:
imdb_df.count()

Title       609
Year        609
Released    608
dtype: int64

In [5]:
#there is a year which can't be cast

imdb_df.loc[imdb_df['Year']=='2007–2011']


,Title,Year,Released
336,Primeval,2007–2011,09 Aug 2008


In [6]:
#take only 2007 from the year above for title: Primeval
imdb_df['Year'] = imdb_df['Year'].str.split('–', 1).str[0]
imdb_df['Year'] = imdb_df['Year'].astype(int)

## Load 2021 Disney Movies Blockbuster Titles

In [7]:
disney_df = pd.read_csv('input/2021_disney_movies_total_gross.csv')
disney_df['year'] = pd.DatetimeIndex(disney_df['release_date']).year
disney_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


In [8]:
disney_df.count()

movie_title                 618
year                        618
release_date                618
total_gross                 618
inflation_adjusted_gross    618
dtype: int64

In [9]:
disney_df = disney_df[['movie_title','year','release_date']].copy()
disney_df['release_date'] = disney_df['release_date'].str.replace(',','')
disney_df.head()

,movie_title,year,release_date
0,101 Dalmatians,1961,Jan 25 1961
1,101 Dalmatians,1996,Nov 27 1996
2,102 Dalmatians,2000,Nov 22 2000
3,1492: Conquest of Paradise,1992,Oct 9 1992
4,"20,000 Leagues Under the Sea",1954,Dec 23 1954


In [10]:
disney_df.dtypes

movie_title     object
year             int64
release_date    object
dtype: object

## Merge 2021 Disney Movies Blockbuster Titles & IMDB Movies Titles

In [11]:
no_imdb_df = pd.merge(disney_df, imdb_df, how='outer', left_on='movie_title', right_on='Title')
no_imdb_df.head()

,movie_title,year,release_date,Title,Year,Released
0,101 Dalmatians,1961.0,Jan 25 1961,101 Dalmatians,1996.0,27 Nov 1996
1,101 Dalmatians,1996.0,Nov 27 1996,101 Dalmatians,1996.0,27 Nov 1996
2,102 Dalmatians,2000.0,Nov 22 2000,102 Dalmatians,2000.0,22 Nov 2000
3,1492: Conquest of Paradise,1992.0,Oct 9 1992,1492: Conquest of Paradise,1992.0,09 Oct 1992
4,"20,000 Leagues Under the Sea",1954.0,Dec 23 1954,"20,000 Leagues Under the Sea",1954.0,20 Jul 1955


In [12]:
no_imdb_df = no_imdb_df.loc[no_imdb_df['Title'].isna()]
no_imdb_df.head()

,movie_title,year,release_date,Title,Year,Released
34,America's Heart and Soul,2004.0,Jul 2 2004,NaN,NaN,NaN
78,Blame it on the Bellboy,1992.0,Mar 6 1992,NaN,NaN,NaN
84,Born in China,2017.0,Apr 21 2017,NaN,NaN,NaN
85,Bound by Honor,1993.0,Apr 16 1993,NaN,NaN,NaN
163,Endurance,1999.0,May 14 1999,NaN,NaN,NaN


In [13]:
export_no_imdb_df=no_imdb_df.dropna(axis=1,how='all')
export_no_imdb_df.count()

movie_title     32
year            32
release_date    32
dtype: int64

In [14]:
export_no_imdb_df.head()

,movie_title,year,release_date
34,America's Heart and Soul,2004.0,Jul 2 2004
78,Blame it on the Bellboy,1992.0,Mar 6 1992
84,Born in China,2017.0,Apr 21 2017
85,Bound by Honor,1993.0,Apr 16 1993
163,Endurance,1999.0,May 14 1999


## Save CSV No IMDB Blockbuster Movies Titles

These titles will be check by hand

In [15]:
#export export_no_imdb_df and run again later when we have the correct list
export_no_imdb_df.to_csv(r'output/no_imdb_movies.csv', index = None, header=True) 

In [16]:
# use no_imdb_df to continue working on the title discrepencies

## ONE DAY LATER

AFTER manually checking for the correct information on IMDB

## Resolved the Name & Year Manually and now ready to be added to the list

In [17]:
fixed_name_df=pd.read_csv('input/no_imdb_movies_FIXED_manually.csv', encoding= 'unicode_escape')
fixed_name_df.head()

,movie_title,year,release_date
0,America's Heart and Soul,2004,Jul 2 2004
1,Blame it on the Bellboy,1992,Mar 6 1992
2,Born in China,2017,Apr 21 2017
3,Bound by Honor,1993,Apr 16 1993
4,Endurance,1998,May 14 1999


In [18]:
# fixed_name_df['movie_title'] = fixed_name_df['movie_title'].str.replace('3 Men and','Three Men and')
fixed_name_df[fixed_name_df.movie_title.str.startswith('Fantasia')]

,movie_title,year,release_date


In [19]:
# #export export_no_imdb_df and run again later when we have the correct list
# fixed_name_df.to_csv(r'input/no_imdb_movies_FIXED_manually.csv', index = None, header=True) 

In [20]:
export_no_imdb_df.rename(columns={'movie_title': 'old_movie_title', 'year': 'old_year',\
                                  'release_date': 'old_release_date'}, inplace=True)
export_no_imdb_df = export_no_imdb_df.reset_index(drop=True)
export_no_imdb_df.head()

/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,old_movie_title,old_year,old_release_date
0,America's Heart and Soul,2004.0,Jul 2 2004
1,Blame it on the Bellboy,1992.0,Mar 6 1992
2,Born in China,2017.0,Apr 21 2017
3,Bound by Honor,1993.0,Apr 16 1993
4,Endurance,1999.0,May 14 1999


In [21]:
resolved_df = pd.concat([export_no_imdb_df, fixed_name_df], axis=1)
resolved_df.head()

,old_movie_title,old_year,old_release_date,movie_title,year,release_date
0,America's Heart and Soul,2004.0,Jul 2 2004,America's Heart and Soul,2004,Jul 2 2004
1,Blame it on the Bellboy,1992.0,Mar 6 1992,Blame it on the Bellboy,1992,Mar 6 1992
2,Born in China,2017.0,Apr 21 2017,Born in China,2017,Apr 21 2017
3,Bound by Honor,1993.0,Apr 16 1993,Bound by Honor,1993,Apr 16 1993
4,Endurance,1999.0,May 14 1999,Endurance,1998,May 14 1999


In [22]:
#fillna for one movie title "crazy/beautiful"
resolved_df['movie_title'].fillna(resolved_df['old_movie_title'], inplace=True)
resolved_df['year'].fillna(resolved_df['old_year'], inplace=True)
resolved_df['release_date'].fillna(resolved_df['old_release_date'], inplace=True)
resolved_df.astype({'year': 'int32'},{'old_year': 'int32'}).dtypes

old_movie_title      object
old_year            float64
old_release_date     object
movie_title          object
year                  int32
release_date         object
dtype: object

In [23]:
resolved_df[resolved_df.movie_title.str.startswith('Crazy')]

,old_movie_title,old_year,old_release_date,movie_title,year,release_date


In [24]:
resolved_df[resolved_df.movie_title.str.startswith('Fantasia')]

,old_movie_title,old_year,old_release_date,movie_title,year,release_date


In [25]:
disney_df = pd.read_csv('output/2021_disney_movies_total_gross.csv')
disney_df['year'] = pd.DatetimeIndex(disney_df['release_date']).year
disney_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


In [26]:
#Merge the Disney disney and resolved dataset on movie_title & year ONLY
merged_df =pd.merge(disney_df, resolved_df, how='left', left_on=['movie_title','year'], right_on=['old_movie_title','old_year'], indicator=True)
merged_df.head()

,movie_title_x,year_x,release_date_x,total_gross,inflation_adjusted_gross,old_movie_title,old_year,old_release_date,movie_title_y,year_y,release_date_y,_merge
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [27]:
merged_df[merged_df['_merge'] == 'both']

,movie_title_x,year_x,release_date_x,total_gross,inflation_adjusted_gross,old_movie_title,old_year,old_release_date,movie_title_y,year_y,release_date_y,_merge
34,America's Heart and Soul,2004,"Jul 2, 2004",314000,450421,America's Heart and Soul,2004.0,Jul 2 2004,America's Heart and Soul,2004.0,Jul 2 2004,both
78,Blame it on the Bellboy,1992,"Mar 6, 1992",2891055,5583671,Blame it on the Bellboy,1992.0,Mar 6 1992,Blame it on the Bellboy,1992.0,Mar 6 1992,both
84,Born in China,2017,"Apr 21, 2017",13873211,15336259,Born in China,2017.0,Apr 21 2017,Born in China,2017.0,Apr 21 2017,both
85,Bound by Honor,1993,"Apr 16, 1993",4496583,8432104,Bound by Honor,1993.0,Apr 16 1993,Bound by Honor,1993.0,Apr 16 1993,both
163,Endurance,1999,"May 14, 1999",229128,372669,Endurance,1999.0,May 14 1999,Endurance,1998.0,May 14 1999,both
181,Firelight,1998,"Sep 4, 1998",784419,1304012,Firelight,1998.0,Sep 4 1998,Firelight,1997.0,Sep 4 1998,both
198,Gedo Senki,2010,"Aug 13, 2010",48658,60465,Gedo Senki,2010.0,Aug 13 2010,Gedo Senki,2010.0,Aug 13 2010,both
199,George Of The Jungle,1997,"Jul 16, 1997",105263257,177714546,George Of The Jungle,1997.0,Jul 16 1997,George Of The Jungle,1997.0,Jul 16 1997,both
202,Gnomeo and Juliet,2011,"Feb 11, 2011",99967670,120424824,Gnomeo and Juliet,2011.0,Feb 11 2011,Gnomeo & Juliet,2011.0,Feb 11 2011,both
204,Goal!,2006,"May 12, 2006",4283255,5757111,Goal!,2006.0,May 12 2006,Goal!,2006.0,May 12 2006,both


In [28]:
for i in merged_df.index: 
     if merged_df['_merge'][i] == 'both':
        print (merged_df["movie_title_x"][i])
        print (merged_df["movie_title_y"][i])
        print (merged_df["year_x"][i])
        print (merged_df["year_y"][i])
        merged_df["movie_title_x"][i] = merged_df["movie_title_y"][i]
        merged_df["year_x"][i] = merged_df["year_y"][i]
        print ('=======================')

America's Heart and Soul
America's Heart and Soul
2004
2004.0
Blame it on the Bellboy
Blame it on the Bellboy
1992
1992.0
Born in China
Born in China
2017
2017.0
Bound by Honor
Bound by Honor
1993
1993.0
Endurance
Endurance
1999
1998.0
Firelight
Firelight
1998
1997.0
Gedo Senki
Gedo Senki
2010
2010.0
George Of The Jungle
George Of The Jungle
1997
1997.0
Gnomeo and Juliet
Gnomeo & Juliet
2011
2011.0
Goal!
Goal!
2006
2006.0
Honey I Blew Up the Kid
Honey I Blew Up the Kid
1992
1992.0
I am Number Four
I am Number Four
2011
2011.0
I'll Be Home For Christmas
I'll Be Home For Christmas
1998
1998.0
Jane Austen's Mafia!
Jane Austen's Mafia!
1998
1998.0
Life With Mikey
Life with Mikey
1993
1993.0
Mars Needs Moms
Mars Needs Moms
2011
2010.0
Nothing To Lose
Nothing To Lose
1997
1997.0
Return to Never Land
Peter Pan 2: Return to Never Land
2002
2002.0
Planes: Fire and Rescue
Planes: Fire and Rescue
2014
2014.0
Play it to the Bone
Play It to the Bone
1999
1999.0
Ralph Breaks The Internet
Ralph Break

/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/vuvie/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
#check to see if movie_title & year are populated correctly
merged_df[merged_df.movie_title_x.str.startswith('Fantasia')]

,movie_title_x,year_x,release_date_x,total_gross,inflation_adjusted_gross,old_movie_title,old_year,old_release_date,movie_title_y,year_y,release_date_y,_merge
171,Fantasia,1940,"Nov 13, 1940",83320000,1612658600,NaN,NaN,NaN,NaN,NaN,NaN,left_only
172,Fantasia 2000,2000,"Jan 1, 2000",60507228,95212796,NaN,NaN,NaN,NaN,NaN,NaN,left_only
173,Fantasia 2000,2000,"Jun 16, 2000",9103630,14325264,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [30]:
#check to see if movie_title & year are populated correctly
merged_df[merged_df.movie_title_x.str.startswith('An')]

,movie_title_x,year_x,release_date_x,total_gross,inflation_adjusted_gross,old_movie_title,old_year,old_release_date,movie_title_y,year_y,release_date_y,_merge
36,An Alan Smithee Film: Burn Hollywood Burn,1998,"Feb 27, 1998",45779,76102,NaN,NaN,NaN,NaN,NaN,NaN,left_only
37,An American Werewolf in Paris,1997,"Dec 25, 1997",26570463,44858556,NaN,NaN,NaN,NaN,NaN,NaN,left_only
38,An Innocent Man,1989,"Oct 6, 1989",19397847,42388988,NaN,NaN,NaN,NaN,NaN,NaN,left_only
39,Angels in the Outfield,1994,"Jul 15, 1994",50236831,91853401,NaN,NaN,NaN,NaN,NaN,NaN,left_only
40,Angie,1994,"Mar 4, 1994",9252349,16917064,NaN,NaN,NaN,NaN,NaN,NaN,left_only
41,Annapolis,2006,"Jan 27, 2006",17127992,23021686,NaN,NaN,NaN,NaN,NaN,NaN,left_only
42,Another Stakeout,1993,"Jul 23, 1993",20107504,37706092,NaN,NaN,NaN,NaN,NaN,NaN,left_only
43,Ant-Man,2015,"Jul 17, 2015",180202163,206016362,NaN,NaN,NaN,NaN,NaN,NaN,left_only
44,Ant-Man and the Wasp,2018,"Jul 6, 2018",216648740,233786031,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [31]:
merged_df.drop(columns=['old_movie_title', 'old_year', 'old_release_date', 'movie_title_y', 'year_y',\
               'release_date_y', '_merge'], inplace=True)

merged_df.rename(columns={'movie_title_x': 'movie_title', 'year_x': 'year',\
                          'release_date_x': 'release_date'}, inplace=True)

merged_df['movie_title'] = merged_df['movie_title'].str.replace('.','')
merged_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


In [32]:
#check to see if movie_title & year are populated correctly
merged_df[merged_df.movie_title.str.startswith('An')]

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
36,An Alan Smithee Film: Burn Hollywood Burn,1998,"Feb 27, 1998",45779,76102
37,An American Werewolf in Paris,1997,"Dec 25, 1997",26570463,44858556
38,An Innocent Man,1989,"Oct 6, 1989",19397847,42388988
39,Angels in the Outfield,1994,"Jul 15, 1994",50236831,91853401
40,Angie,1994,"Mar 4, 1994",9252349,16917064
41,Annapolis,2006,"Jan 27, 2006",17127992,23021686
42,Another Stakeout,1993,"Jul 23, 1993",20107504,37706092
43,Ant-Man,2015,"Jul 17, 2015",180202163,206016362
44,Ant-Man and the Wasp,2018,"Jul 6, 2018",216648740,233786031


In [33]:
#export the FINAL correct list
merged_df.to_csv(r'output/FINAL_2021_disney_movies_total_gross.csv', index = None, header=True) 

## Save new list for delta to scrape again

In [34]:
# #save delta movie lists for scraping again

# delta_df = merged_df[merged_df['_merge'] == 'both']
# delta_df.count()

In [35]:
# delta_df.drop(columns=['old_movie_title', 'old_year', 'old_release_date', 'movie_title_y', 'year_y',\
#                'release_date_y', '_merge'], inplace=True)

# delta_df.rename(columns={'movie_title_x': 'movie_title', 'year_x': 'year',\
#                           'release_date_x': 'release_date'}, inplace=True)

# delta_df['movie_title'] = delta_df['movie_title'].str.replace('.','')

# delta_df.head()

In [36]:
# #export the FINAL correct list
# delta_df.to_csv(r'output/delta_2021_disney_movies_total_gross.csv', index = None, header=True) 